# Aprendizaje de Máquina - Tarea 6

**Alumno:** César Zamora Martínez
 **Fecha:** 19 de Noviembre de 2019 

### Ejercicio 1.3 Redes de múltiples capas para clasificación

Demuestre que
$$ \nabla_{ a^{(L+1)} (x)} l(f (\mathbf{x}), y) = − (e(y) − f (x)) $$
Donde $e(y) \in  \mathbb{R}$ m es un vector tal que $e(y)_i = 1_{y = i}$.

**Pendiente: Desarrollo**

Para ello debemos notar que $$\mathcal{l}(f(x),y) =  -\log(f(x)_y)  =  -\log(softmax(y, a^{(L+1)}(x))) = \log( \sum_i exp({a^{(L+1)} (x)_i}) - a^{(L+1)}(x)_y  $$

De dicha observación se desprende que:

$$ \frac{ \partial }{ \partial a^{(L+1)} (x)_i } \mathcal{l}(f(x),y) = \begin{cases} \frac{ exp({a^{(L+1)} (x)_y}) }{\sum_i exp({a^{(L+1)} (x)_i})} - 1 & i=y \\ \frac{ exp({a^{(L+1)} (x)_y}) }{\sum_i exp({a^{(L+1)} (x)_i})}  & i\neq y \end{cases}$$

$$ =  \begin{cases} softmax(y, a^{(L+1)}(x)) + 1 & i=y \\ softmax(y, a^{(L+1)}(x))  & i\neq y \end{cases} $$

$$ =  softmax(y, a^{(L+1)}(x)) + 1_{y = i} = softmax(y, a^{(L+1)}(x))- e(y) =   − (e(y) − f(\mathbf{x})) $$

## 2 Clasificación con MNIST
 
### Ejercicio 2.4

El archivo correspondiente a la implementación  se denominó **modelos.py**, que como parte de si despliega la siguiente implementación. **Nota:** el código propuesto en la tarea no contiene ningún marcador de tipo comentario, sin embargo se han añadido comentarios para describir su funcionamiento:

In [ ]:
import torch.nn as nn

# ==================
# Regresion logistica multinomial
# ==================
class RegresionMultinomial(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RegresionMultinomial, self).__init__() # Metodo para que la clase RegresionMultinomial herede atributos y metodos de clases hijas de nn.Module
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        out = self.linear(x)
        return out

### Ejercicio 2.5
a) Llene las líneas de [COMENTARIO] indicadas.

A continuación se presenta la implementación del programa **regresion_multinomial.py**, que contiene la implementación para el método de regresión logística multinomial incorporando comentarios que precisar su funcionamiento:

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from modelos import RegresionMultinomial

# Permite realizar emplear una tarjeta GPU de Nvida a traves de CUDA
# En caso de que el equipo no cuente con tal hardware, el proceso de 
# estimacion se lleva a cabo en el cpu del equipo huesped
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hiper-parametros
input_size = 784 # Dimension de datos de entrada (28 x 28)
num_classes = 10 # MNIST tiene 10 clases (numeros del 1 al 10)
num_epochs = 5 # Numero de epocas para entrenar
bs = 100 # Tamano de lote (batch_size)
lr = 0.001 # Tasa de aprendizaje

# Cadena de texto para indicar el directorio en donde se guardaran los
# datos en lo que se basara el modelo de prediccion (es decir,
# tanto datos en crudo como los resultados del procesamiento). 
# La carpeta se creo previamente con el comandor mkdir
root = './datos'
# Carpeta donde se guardaran los datos
train_data = MNIST(root, train=True, transform=ToTensor(), download=True)
test_data = MNIST(root, train=False, transform=ToTensor())

# Se cargan los datos que serviran como entrenamiento y prueba empleando
# la funcion DataLoader. Esta permite realizar cargas por lotes de un 
# tamano especifico (batch_size), indicando si se emplean o no un metodo
# de reemplazo (shuffle), de acuerdo con la documentacion:
# https://pytorch.org/docs/stable/_modules/torch/utils/data/dataloader.html

train_loader = DataLoader(train_data, batch_size=bs, shuffle=True)
test_loader = DataLoader(test_data, batch_size=bs, shuffle=False)

# ==================
# Definimos modelo
# ==================
# Instancia la clase del modelo de red a modelar (importada del archivo
# models.py), junto con el tamano de sus datos de entrada y numero de clases
# para el problema de clasificacion. En adicion, se establece el tipo de funcion
#  de perdida que se considerara en el problema de prediccion para establecer
# la funcion de riesgo (empirico)

# En este caso, se emplea  un modelo de regresion multinomial 
# junto con la implementacion de PyTorch de la entropia cruzada

model = RegresionMultinomial(input_size, num_classes).to(device)
loss_function = nn.CrossEntropyLoss()

# ==================
# Optimizacion
# ==================
# Metodo con el que se realiza la optimizacion de los parametros de la red
# Es este caso de emplea la imlpementacion del descenso de gradiente estocastico
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
# Entrenamiento del modelo
for epoch in range(num_epochs):
    for i, (xi, yi) in enumerate(train_loader):

        # Las imagenes y correspondientes etiquetas para su clasificacion en el conjunto de
	# entrenamiento se envian con cuda al dipositivo GPU  correspondiente, 
	#o en caso contrario se procesan en el CPU del equipo huesped
        xi = xi.reshape(-1, 28*28).to(device) # imagenes
        yi = yi.to(device) # etiquetas

        # Propagacion para adelante
        output = model(xi)
        loss = loss_function(output, yi)

        # Propagacion para atras y paso de optimizacion
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoca: {}/{}, Paso: {}/{}, Perdida: {:.5f}'.format(epoch+1,num_epochs, i+1, len(train_loader), loss.item()))


# Prueba del modelo
# Al probar, usamos torch.no_grad() porque le decimos autograd que no guarde registro de 
# estas operaciones, al no formar parte del proceso de optimización del modelo, sino 
# de probar su nivel de prediccion
# Ver: https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#gradients

with torch.no_grad():
    correct = 0
    total = 0

    for xi, yi in test_loader:

        # Nuevamente, imagenes y correspondientes etiquetas para su clasificacion
	# pertenecientes al conjunto de prueba se envian con cuda al dipositivo
	# GPU  correspondiente, o en caso contrario
        # se procesan en el CPU del equipo huesped

        xi = xi.reshape(-1,28*28).to(device)
        yi = yi.to(device)

        # Con el modelo entrenado, se predice la etiqueta de cada imagen xi en el conjunto
	# de prueba, y se cuente para cuantos de estos la etiqueta predecida corresponde
	# con el valor real. Dicha variable de calcula de manera iterativa para imprimir
	# finalmente el porcentaje de etiquetas correctamente predichas por el modelo.
        output = model(xi)
        _, predicted = torch.max(output.data, 1)

        total += yi.size(0)
        correct += (predicted == yi).sum().item()

    print(f'Precision del modelo en {total} imagenes: {100 * correct / total}')


# Permite  escibrir en disco (a traves de un diccionario) al modelo que se 
# establecido con los datos cargados en un archivo de formato .ckpt denomiado "modelo"
save_model = False
if save_model is True:
    torch.save(model.state_dict(), 'modelo.ckpt')

**Nota:**

Antes de proseguir en los argumentos de solución a esta tarea, cabe destacar que a efecto de extender el código proporcionado para las redes recié presentadas para todas las redes que se abordan en esta tarea, al código de **modelos.py** se añadieron clases que permiten representar la regresión logística multinomila bajo los métodos de optimización SGD y Adam, así como las siguientes redes 1) RedNeuronal: Una capa escondida con 500 unidades (la que ya tenemos),  2) RedNeuronal3: Tres capas escondidas de 500, 100 y 30 unidades, 3) RedNeuronal5: Cinco capas escondidas de 500, 300, 100, 50, 30 unidades, considerando las posibles variaciones de estas al combinarlas con funciones de activación de tipo sigmoide y ReLU, así como cambiando los métodos de optimización empleados para poder escoger entre SGD y Adam.

Al respecto, la ampliación del archivo **modelos.py** es la siguiente:

In [ ]:
import torch.nn as nn
# ==================
# Regresion logistica multinomial
# ==================
class RegresionMultinomial(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RegresionMultinomial, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        out = self.linear(x)
        return out

# ==================
# Red neuronal de 1 capa escondida con activaciones Sigm
# ==================
class RedNeuronal_Sigm(nn.Module):
    def __init__(self, input_size, num_classes):
        # Permite que la clase  RedNeuronal herede los atributos y metodos de
        # las clases hijas de la clase nn.Module al ser construida
        super(RedNeuronal_Sigm, self).__init__()

        # unidad de capa escondida
        hidden_size = 500

        # funciones de pre-activacion y activacion en capa escondida (linea-sigmoide-lineal)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.sigmoid(out)
        out = self.fc2(out)
        return out

# ==================
# Red neuronal de 1 capa escondida con activaciones ReLU
# ==================
class RedNeuronal_ReLU(nn.Module):
    def __init__(self, input_size, num_classes):
        # Permite que la clase  RedNeuronal herede los atributos y metodos de
        # las clases hijas de la clase nn.Module al ser construida
        super(RedNeuronal_ReLU, self).__init__()

        # unidad de capa escondida
        hidden_size = 500

        # funciones de pre-activacion y activacion en capa escondida (linea-sigmoide-lineal)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out 
    
# ==================
# Red neuronal de 3 capas escondidas con activaciones Sigm
# ==================
class RedNeuronal3_Sigm(nn.Module):
    def __init__(self, input_size, num_classes):
        # Permite que la clase  RedNeuronal herede los atributos y metodos de
        # las clases hijas de la clase nn.Module al ser construida
        super(RedNeuronal3_Sigm, self).__init__()

        # unidades de capas escondidas
        hidden_size1, hidden_size2, hidden_size3  = 500, 100, 30

        # funciones de pre-activacion y activacion en capa escondida (linea-sigmoide-lineal)
        self.fc1= nn.Linear(input_size, hidden_size1)
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.sigmoid(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        out = self.fc4(out)
        return out
    
    
# ==================
# Red neuronal de 3 capas escondidas con activaciones ReLU
# ==================
class RedNeuronal3_ReLU(nn.Module):
    def __init__(self, input_size, num_classes):
        # Permite que la clase  RedNeuronal herede los atributos y metodos de
        # las clases hijas de la clase nn.Module al ser construida
        super(RedNeuronal3_ReLU, self).__init__()

        # unidades de capas escondidas
        hidden_size1, hidden_size2, hidden_size3  = 500, 100, 30

        # funciones de pre-activacion y activacion en capa escondida (linea-sigmoide-lineal)
        self.fc1= nn.Linear(input_size, hidden_size1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        return out

# ==================
# Red neuronal de 5 capas escondidas con activaciones Sigm
# ==================
class RedNeuronal5_Sigm(nn.Module):
    def __init__(self, input_size, num_classes):
    # Permite que la clase  RedNeuronal herede los atributos y metodos de
    # las clases hijas de la clase nn.Module al ser construida
        super(RedNeuronal5_Sigm, self).__init__()

        # unidades de capas escondidas
        hidden_size1, hidden_size2, hidden_size3,hidden_size4, hidden_size5  = 500, 300, 100, 50, 30

        # funciones de pre-activacion y activacion en capa escondida
        self.fc1, self.sigmoid = nn.Linear(input_size, hidden_size1), nn.Sigmoid()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, hidden_size4)
        self.fc5 = nn.Linear(hidden_size4,  hidden_size5)
        self.fc6 = nn.Linear(hidden_size5, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.sigmoid(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        out = self.fc5(out)
        out = self.sigmoid(out)
        out = self.fc6(out)
        return out

# ==================
# Red neuronal de 5 capas escondidas con activaciones ReLU
# ==================
class RedNeuronal5_ReLU(nn.Module):
    def __init__(self, input_size, num_classes):
    # Permite que la clase  RedNeuronal herede los atributos y metodos de
    # las clases hijas de la clase nn.Module al ser construida
        super(RedNeuronal5_ReLU, self).__init__()

        # unidades de capas escondidas
        hidden_size1, hidden_size2, hidden_size3,hidden_size4, hidden_size5  = 500, 300, 100, 50, 30

        # funciones de pre-activacion y activacion en capa escondida
        self.fc1, self.relu = nn.Linear(input_size, hidden_size1), nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, hidden_size4)
        self.fc5 = nn.Linear(hidden_size4,  hidden_size5)
        self.fc6 = nn.Linear(hidden_size5, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        out = self.relu(out)
        out = self.fc5(out)
        out = self.relu(out)
        out = self.fc6(out)
        return out

El código anterior, se usó como base para poder definir una función que nos permite cambiar fácilmente el tipo de red a considerar (de entre una regresión logística multinomial, una red de una capa, un red de tres capas o una de 5), variando la función de activación (entre una de tipo sigmoide o una ReLU), así como el método de optimización a emplear (basado en SGD o Adam de Pytorch).

Al respecto, también se incorporó al método SummaryWriter de Tensorboard de manera que pudiera guardar los archivos de las simulaciones a realizarse y generar dashboard para posterior análisis, quedano como sigue:


#### Importamos el conjunto general de librerias a emplear

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision.datasets import MNIST # Dataset de digitos
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch.utils.tensorboard import SummaryWriter # Utilidad de tensorboard para generar graficos

##### Importamos el archivo donde se definieron los modelos de redes

In [2]:
from modelos import *

#### Función para correr los diferentes modelos de redes

In [4]:
def RunModel(modelo, activacion, optimization_metod, directorio="runs/testing", saving_model = False):
    '''    
    modelo:str red neuronal a considerar ("RN1","RN3","RN5",Reg. Lin. Multi:"RLM")        
    activacion:str funcion de activacion de las capas de red neuronal a considerar
    ("Sigm": funcion sigmoide, "ReLU": funcion ReLU)
    optimization_metod:str Metodo de optimizacion para estimar parametros de la red
        ("SGD": descenso de gradiente estocatico, "Adam": metodo Adam)
    directorio:str especificacion del directorio para que tensorboard guarde datos para dashboards
    saving_model:Bool especifica si el modelo se debe guardar tras correr simulacion
    '''
    
    # Directorio donde se guardaran objetos de tensorboard para generar dashboard de variables monitoreadas
    writer = SummaryWriter(directorio)
    running_loss=0

    #
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Hiper-parametros
    input_size = 784  # Dimension de datos de entrada (28 x 28)
    num_classes = 10  # MNIST tiene 10 clases (numeros del 1 al 10)
    num_epochs = 5  # Numero de epocas para entrenar
    bs = 100  # Tamano de lote (batch_size)
    lr = 0.001  # Tasa de aprendizaje

    #
    root = './datos'
    # Carpeta donde se guardaran los datos
    train_data = MNIST(root, train=True, transform=ToTensor(), download=True)
    test_data = MNIST(root, train=False, transform=ToTensor())

    #
    train_loader = DataLoader(train_data, batch_size=bs, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=bs, shuffle=False)

    # ==================
    # Definimos modelo
    # ==================
    # Definicion del modelo de red
    if modelo == "RLM":    
        model = RegresionMultinomial(input_size, num_classes).to(device)
    if modelo == "RN1"  and activacion == "ReLU":    
        model = RedNeuronal_ReLU(input_size, num_classes).to(device)
    if modelo == "RN1"  and activacion == "Sigm":    
        model = RedNeuronal_Sigm(input_size, num_classes).to(device)
    if modelo == "RN3"  and activacion == "ReLU":    
        model = RedNeuronal3_ReLU(input_size, num_classes).to(device)
    if modelo == "RN3"  and activacion == "Sigm":    
        model = RedNeuronal3_Sigm(input_size, num_classes).to(device)
    if modelo == "RN5"  and activacion == "ReLU":    
        model = RedNeuronal5_ReLU(input_size, num_classes).to(device)
    if modelo == "RN5"  and activacion == "Sigm":    
        model = RedNeuronal5_Sigm(input_size, num_classes).to(device)
        
    loss_function = nn.CrossEntropyLoss()

    # ==================
    # Optimizacion
    # ==================
    # 
    if optimization_metod=="SGD":
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    if optimization_metod=="Adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        
    # Entrenamiento del modelo
    for epoch in range(num_epochs):
        for i, (xi, yi) in enumerate(train_loader):

            # 
            xi = xi.reshape(-1, 28*28).to(device)  # imagenes
            yi = yi.to(device)  # etiquetas

            # Propagacion para adelante
            output = model(xi)
            loss = loss_function(output, yi)

            # Propagcion para atras y paso de optimizacion
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss = loss.item()
            if (i+1) % 100 == 0:
                writer.add_scalar('training loss',running_loss/1000, epoch*len(train_loader)+1)
                print('Epoca: {}/{}, Paso: {}/{}, Perdida: {:.5f}'.format(epoch +
                                                                          1, num_epochs, i+1, len(train_loader), loss.item()))

    # Prueba del modelo
    # Al probar, usamos torch.no_grad()
    with torch.no_grad():
        correct = 0
        total = 0

        for xi, yi in test_loader:
            # 
            xi = xi.reshape(-1, 28*28).to(device)
            yi = yi.to(device)

            # 
            output = model(xi)
            _, predicted = torch.max(output.data, 1)

            total += yi.size(0)
            correct += (predicted == yi).sum().item()

            writer.add_scalar('precision_modelo',correct/100, total)

        print(f'Precision del modelo en {total} imagenes: {100 * correct / total}')


    # 
    save_model = saving_model
    if save_model is True:
        torch.save(model.state_dict(), 'modelo.ckpt')
    
    return 

b) ¿Cuál es el error de clasificación?

Para responder esta pregunta, la función recién presentada se usó para representar un modelo de regresión logística de tipo multinomial, cuyo resultados se presentan a continuación:

c) Explique por qué este código implementa el modelo de regresión logística multinomial.

De acuerdo a lo visto en la sección 1.3 de la tarea, el modelo de regresión en comento incorporar como función de pérdida a la log-verosimilitud negativa cuya expresión es la de la entropía cruzada

$$H(p, q) = −E p [log q] \mbox{ con } p(c) = 1{y = c} \mbox{ y } q(c) = Pr[y = c | x, W]$$.

En este sentido, de acuerdo a la documentación de Pytorch la función **CrossEntropyLoss** (ver https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss) resulta en la implementación de la función de pérdida de entropía cruza, con lo cual la función de riesgo que se asocia al modelo de la red bajo esta herramienta de PyTorch es justamente la función de pérdida del problema de regresión logística de tipo multinomial.


#### Regresión logística multinomial + SGD

In [17]:
RunModel("RLM", "Sigm", "SGD", 'runs/RLM_SGD', saving_model = False)

Epoca: 1/5, Paso: 100/600, Perdida: 2.23107
Epoca: 1/5, Paso: 200/600, Perdida: 2.09345
Epoca: 1/5, Paso: 300/600, Perdida: 2.03219
Epoca: 1/5, Paso: 400/600, Perdida: 1.89934
Epoca: 1/5, Paso: 500/600, Perdida: 1.89142
Epoca: 1/5, Paso: 600/600, Perdida: 1.78481
Epoca: 2/5, Paso: 100/600, Perdida: 1.71872
Epoca: 2/5, Paso: 200/600, Perdida: 1.65567
Epoca: 2/5, Paso: 300/600, Perdida: 1.56736
Epoca: 2/5, Paso: 400/600, Perdida: 1.53417
Epoca: 2/5, Paso: 500/600, Perdida: 1.54434
Epoca: 2/5, Paso: 600/600, Perdida: 1.48852
Epoca: 3/5, Paso: 100/600, Perdida: 1.34137
Epoca: 3/5, Paso: 200/600, Perdida: 1.38734
Epoca: 3/5, Paso: 300/600, Perdida: 1.39153
Epoca: 3/5, Paso: 400/600, Perdida: 1.33284
Epoca: 3/5, Paso: 500/600, Perdida: 1.29637
Epoca: 3/5, Paso: 600/600, Perdida: 1.28410
Epoca: 4/5, Paso: 100/600, Perdida: 1.33315
Epoca: 4/5, Paso: 200/600, Perdida: 1.23097
Epoca: 4/5, Paso: 300/600, Perdida: 1.17208
Epoca: 4/5, Paso: 400/600, Perdida: 1.21201
Epoca: 4/5, Paso: 500/600, Perdi

El error de predicción se puede calcular como $(100- 82.85)/100 \times 100 \% = 17.15\%$

### Ejercicio 2.6

Incluya en su archivo modelos.py la siguiente clase que implementa una red neuronal con una capa escondida y activaciones sigmoide. Rellene los comentarios;


La implementación realizada junto con los comentarios correspondientes se pueden consultar en la ampliación del código de **modelos.py** vista en el ejercicio **2.5**.

### Ejercicio 2.7

Pruebe el desempeño de RedNeuronal y compare los resultados con RegresionMultinomial.

Al respecto, la implementación correspondiente se refiere al uso de la función presentada en 2.5 para laed Neuronal de una capa escondida, con activación de tipo sigmoide y método de optimización SGD, cuyos resultados se presentan en seguida:

#### Red Neuronal de una capa escondida + Activaciones Sigmoide + SGD

In [19]:
RunModel("RN1", "Sigm", "SGD", 'runs/RN1_Sigm_SGD', saving_model = False)

Epoca: 1/5, Paso: 100/600, Perdida: 2.31708
Epoca: 1/5, Paso: 200/600, Perdida: 2.29881
Epoca: 1/5, Paso: 300/600, Perdida: 2.29079
Epoca: 1/5, Paso: 400/600, Perdida: 2.28210
Epoca: 1/5, Paso: 500/600, Perdida: 2.27837
Epoca: 1/5, Paso: 600/600, Perdida: 2.28610
Epoca: 2/5, Paso: 100/600, Perdida: 2.27670
Epoca: 2/5, Paso: 200/600, Perdida: 2.27217
Epoca: 2/5, Paso: 300/600, Perdida: 2.27454
Epoca: 2/5, Paso: 400/600, Perdida: 2.27046
Epoca: 2/5, Paso: 500/600, Perdida: 2.26060
Epoca: 2/5, Paso: 600/600, Perdida: 2.25681
Epoca: 3/5, Paso: 100/600, Perdida: 2.25576
Epoca: 3/5, Paso: 200/600, Perdida: 2.23885
Epoca: 3/5, Paso: 300/600, Perdida: 2.23969
Epoca: 3/5, Paso: 400/600, Perdida: 2.24974
Epoca: 3/5, Paso: 500/600, Perdida: 2.22979
Epoca: 3/5, Paso: 600/600, Perdida: 2.24076
Epoca: 4/5, Paso: 100/600, Perdida: 2.22582
Epoca: 4/5, Paso: 200/600, Perdida: 2.22268
Epoca: 4/5, Paso: 300/600, Perdida: 2.22946
Epoca: 4/5, Paso: 400/600, Perdida: 2.22880
Epoca: 4/5, Paso: 500/600, Perdi

De manera análoga al ejercicio anterior, ello implica un error de predicción de 55.89%, el cual es casi tres veces mayor al obtenido en el caso de la regresión logística multinomial presentando previamente.

### Ejercicio 2.8

a) Genere gráficas que muestren el decremento en la función de pérdida y en la precisión sobre los valores de prueba (Puede usar matplotlib o tensorboard). Comente sus observaciones.

Para dar respuesta a este ejercicio, se corrieron el resto de modelos correspondientes a las redes RedNeuronal, RedNeuronal3 y RedNeuronal5, variando el método de optimización junto con la función de activación. Los resultados obtenidos se pueden revisar en el Anexo al final del presente documento.

Ahora bien, a efecto de falicitar el entendimiento de los siguientes gráficos, se presenta una tabla que resume la notación empleada para nombrar cada modelo de red neuronal, según su tipo, función de activación y método de optimizacion:

| Modelo             | Activaciones | Optimizador | Notación |
|--------------------|--------------|-----------------|------------------|
| RedNeuronal        | ReLU         | SGD       |      RN_ReLU_SGD      |
| RedNeuronal        | ReLU         | Adam      |       RN_ReLU_Adam    |
| RedNeuronal        | Sigmoid      | SGD       |     RN_Sigm_SGD       |
| RedNeuronal        | Sigmoid      | Adam      |       RN_Sigm_Adam    |
| RedNeuronal3        | ReLU         | SGD       |      RN3_ReLU_SGD      |
| RedNeuronal3        | ReLU         | Adam      |       RN3_ReLU_Adam    |
| RedNeuronal3        | Sigmoid      | SGD       |     RN3_Sigm_SGD       |
| RedNeuronal3        | Sigmoid      | Adam      |       RN3_Sigm_Adam    |
| RedNeuronal5        | ReLU         | SGD       |      RN5_ReLU_SGD      |
| RedNeuronal5       | ReLU         | Adam      |       RN5_ReLU_Adam    |
| RedNeuronal5        | Sigmoid      | SGD       |     RN5_Sigm_SGD       |
| RedNeuronal5        | Sigmoid      | Adam      |       RN5_Sigm_Adam    |

**Pérdidas a lo largo de las épocas**

A continuación se muestran las gráficas la perdida obtenidas a lo largo de las diferentes épocas. Se puede apreciar que el modelo RN5\_ReLU\_Adam fue el que observó un pérdida más baja en las iteraciones consideradas:

![Error de entrenamiento a lo largo de las épocas](images/loss.png)
![Error de entrenamiento a lo largo de las épocas](images/loss_table.png)

Cabe apreciar que los modelos de la RedNeuronal basados en SGD tuvieron las pérdidas más bajas.

**Precisión del modelo lo largo de las épocas**

A continuación se muestra las gráficas la precisiṕn obtenida pr el modelo al evaluar la predicción de cada elemento del conjunto de prueba contra el dígito correspondiente. Se puede apreciar que el modelo RN1\_ReLU\_Adam fue con la mejor precisión (97.94%):

![Error de entrenamiento a lo largo de las épocas](images/precision.png)
![Error de entrenamiento a lo largo de las épocas](images/precision_table.png)

En adición, cabe apreciar que los models de RedNeuronal3 y RedNeuronal5 basados función de activación sigmoide y método de optimización SGD tuvieron la peor predicción con 11.35%.

Para complementar este punto, a continuación se presenta un tabla resumen de los valores obtenidos para la precisión en los distintos modelos:

**Precisión obtenida con los diferentes modelos**

| Modelo             | Activaciones | torch.optim.SGD | torch.optim.Adam |
|--------------------|--------------|-----------------|------------------|
| RegresionLogistica | -            | 82.85%           | 92.26%            |
| RedNeuronal        | ReLU         | 44.11%          | 96.75%            |
| RedNeuronal        | Sigmoid      | 78.66%           | 97.74%            |
| RedNeuronal3       | ReLU         | 11.35%           | 97.27%            |
| RedNeuronal3       | Sigmoid      | 35.21%           | 97.76%            |
| RedNeuronal5       | ReLU         | 11.35%           | 97.02%            |
| RedNeuronal5       | Sigmoid      | 11.35%           | 97.47%            |

Se puede observar como, en términos generales, los modelos de redes neuronales basados en el método de optimización Adam arrojan una precisión alta para este problema de clasificación, independientemente de la función de activación considerada, en contraste con aquellos basados en SGD.

b) Complete las siguientes tablas.

**Error de predicción**


|Modelo             | Activaciones | torch.optim.SGD | torch.optim.Adam |
|--------------------|--------------|-----------------|------------------|
| RegresionLogistica | -            | 17.15%           | 7.74%             |
| RedNeuronal        | ReLU         | 57.89%           | 3.25%             |
| RedNeuronal        | Sigmoid      | 21.34%           | 2.26%             |
| RedNeuronal3       | ReLU         | 88.65%           | 2.73%             |
| RedNeuronal3       | Sigmoid      | 64.79%           | 2.24%             |
| RedNeuronal5       | ReLU         | 88.65%           | 2.98%             |
| RedNeuronal5       | Sigmoid      | 88.65%           | 2.53%             |

### Ejercicio 2.9. 

¿Qué puede concluir a partir de estos experimentos?

Derivado del ejercicio anterior, se pueden presumir los siguientes puntos:

* En este problema particular de clasificación, aparentemente Adam es una mejor opción de método de optimización de SGD pues arroja errores de predicción muy bajos al compararse con los obtenidos contra métodos basados en la misma red pero que usan SGD.
* Asimismo, la red predice mejor el problema de clasificación de imágenes de dígitos cuando se emplean ReLU que cuando se echa mano de Sigmoid, restringidos al optimizador SGD.
* Dicha tendencia se invierte si el método de optimización es Adam.
* El desempeño de la predicción con regresión logística multinomial en superado por más de la mitad por los métodos basados en redes neuronales en aquellos casos en los que se usa a Adam como optimizador de los parámetros de la red.

## Anexo de resultados de resto de modelos:

In [18]:
RunModel("RLM", "Sigm", "Adam", 'runs/RLM_Adam', saving_model = False)

Epoca: 1/5, Paso: 100/600, Perdida: 0.84714
Epoca: 1/5, Paso: 200/600, Perdida: 0.55226
Epoca: 1/5, Paso: 300/600, Perdida: 0.43952
Epoca: 1/5, Paso: 400/600, Perdida: 0.58421
Epoca: 1/5, Paso: 500/600, Perdida: 0.37473
Epoca: 1/5, Paso: 600/600, Perdida: 0.44304
Epoca: 2/5, Paso: 100/600, Perdida: 0.44089
Epoca: 2/5, Paso: 200/600, Perdida: 0.43384
Epoca: 2/5, Paso: 300/600, Perdida: 0.36927
Epoca: 2/5, Paso: 400/600, Perdida: 0.28155
Epoca: 2/5, Paso: 500/600, Perdida: 0.23258
Epoca: 2/5, Paso: 600/600, Perdida: 0.31027
Epoca: 3/5, Paso: 100/600, Perdida: 0.26544
Epoca: 3/5, Paso: 200/600, Perdida: 0.41425
Epoca: 3/5, Paso: 300/600, Perdida: 0.41398
Epoca: 3/5, Paso: 400/600, Perdida: 0.28983
Epoca: 3/5, Paso: 500/600, Perdida: 0.31366
Epoca: 3/5, Paso: 600/600, Perdida: 0.31253
Epoca: 4/5, Paso: 100/600, Perdida: 0.42190
Epoca: 4/5, Paso: 200/600, Perdida: 0.39038
Epoca: 4/5, Paso: 300/600, Perdida: 0.39345
Epoca: 4/5, Paso: 400/600, Perdida: 0.33981
Epoca: 4/5, Paso: 500/600, Perdi

#### Regresión logística multinomial + Adam

### Red Neuronal de una capa

#### Red Neuronal de una capa escondida + Activaciones ReLU + SGD

In [6]:
RunModel("RN1", "ReLU", "SGD", 'runs/RN1_ReLU_SGD', saving_model = False)

Epoca: 1/5, Paso: 100/600, Perdida: 2.27098
Epoca: 1/5, Paso: 200/600, Perdida: 2.25274
Epoca: 1/5, Paso: 300/600, Perdida: 2.21254
Epoca: 1/5, Paso: 400/600, Perdida: 2.20317
Epoca: 1/5, Paso: 500/600, Perdida: 2.18718
Epoca: 1/5, Paso: 600/600, Perdida: 2.17403
Epoca: 2/5, Paso: 100/600, Perdida: 2.10185
Epoca: 2/5, Paso: 200/600, Perdida: 2.10962
Epoca: 2/5, Paso: 300/600, Perdida: 2.01843
Epoca: 2/5, Paso: 400/600, Perdida: 2.05426
Epoca: 2/5, Paso: 500/600, Perdida: 2.01789
Epoca: 2/5, Paso: 600/600, Perdida: 1.98587
Epoca: 3/5, Paso: 100/600, Perdida: 1.94279
Epoca: 3/5, Paso: 200/600, Perdida: 1.93208
Epoca: 3/5, Paso: 300/600, Perdida: 1.88867
Epoca: 3/5, Paso: 400/600, Perdida: 1.87086
Epoca: 3/5, Paso: 500/600, Perdida: 1.87709
Epoca: 3/5, Paso: 600/600, Perdida: 1.78198
Epoca: 4/5, Paso: 100/600, Perdida: 1.82296
Epoca: 4/5, Paso: 200/600, Perdida: 1.69206
Epoca: 4/5, Paso: 300/600, Perdida: 1.70763
Epoca: 4/5, Paso: 400/600, Perdida: 1.61053
Epoca: 4/5, Paso: 500/600, Perdi

#### Red Neuronal de una capa escondida + Activaciones Sigmoide + Adam

In [7]:
RunModel("RN1", "Sigm", "Adam", 'runs/RN1_Sigm_Adam', saving_model = False)

Epoca: 1/5, Paso: 100/600, Perdida: 0.63190
Epoca: 1/5, Paso: 200/600, Perdida: 0.24995
Epoca: 1/5, Paso: 300/600, Perdida: 0.38825
Epoca: 1/5, Paso: 400/600, Perdida: 0.22471
Epoca: 1/5, Paso: 500/600, Perdida: 0.31148
Epoca: 1/5, Paso: 600/600, Perdida: 0.23046
Epoca: 2/5, Paso: 100/600, Perdida: 0.25828
Epoca: 2/5, Paso: 200/600, Perdida: 0.14849
Epoca: 2/5, Paso: 300/600, Perdida: 0.20679
Epoca: 2/5, Paso: 400/600, Perdida: 0.22641
Epoca: 2/5, Paso: 500/600, Perdida: 0.22358
Epoca: 2/5, Paso: 600/600, Perdida: 0.13848
Epoca: 3/5, Paso: 100/600, Perdida: 0.13382
Epoca: 3/5, Paso: 200/600, Perdida: 0.20473
Epoca: 3/5, Paso: 300/600, Perdida: 0.07089
Epoca: 3/5, Paso: 400/600, Perdida: 0.16958
Epoca: 3/5, Paso: 500/600, Perdida: 0.10779
Epoca: 3/5, Paso: 600/600, Perdida: 0.17531
Epoca: 4/5, Paso: 100/600, Perdida: 0.07445
Epoca: 4/5, Paso: 200/600, Perdida: 0.06639
Epoca: 4/5, Paso: 300/600, Perdida: 0.15601
Epoca: 4/5, Paso: 400/600, Perdida: 0.11596
Epoca: 4/5, Paso: 500/600, Perdi

#### Red Neuronal de una capa escondida + Activaciones ReLU + Adam

In [8]:
RunModel("RN1", "ReLU", "Adam", 'runs/RN1_ReLU_Adam', saving_model = False)

Epoca: 1/5, Paso: 100/600, Perdida: 0.40839
Epoca: 1/5, Paso: 200/600, Perdida: 0.22708
Epoca: 1/5, Paso: 300/600, Perdida: 0.34032
Epoca: 1/5, Paso: 400/600, Perdida: 0.13182
Epoca: 1/5, Paso: 500/600, Perdida: 0.29857
Epoca: 1/5, Paso: 600/600, Perdida: 0.13923
Epoca: 2/5, Paso: 100/600, Perdida: 0.09076
Epoca: 2/5, Paso: 200/600, Perdida: 0.08429
Epoca: 2/5, Paso: 300/600, Perdida: 0.04753
Epoca: 2/5, Paso: 400/600, Perdida: 0.08848
Epoca: 2/5, Paso: 500/600, Perdida: 0.06071
Epoca: 2/5, Paso: 600/600, Perdida: 0.09700
Epoca: 3/5, Paso: 100/600, Perdida: 0.07444
Epoca: 3/5, Paso: 200/600, Perdida: 0.07097
Epoca: 3/5, Paso: 300/600, Perdida: 0.09218
Epoca: 3/5, Paso: 400/600, Perdida: 0.06149
Epoca: 3/5, Paso: 500/600, Perdida: 0.07703
Epoca: 3/5, Paso: 600/600, Perdida: 0.07854
Epoca: 4/5, Paso: 100/600, Perdida: 0.03885
Epoca: 4/5, Paso: 200/600, Perdida: 0.01693
Epoca: 4/5, Paso: 300/600, Perdida: 0.04746
Epoca: 4/5, Paso: 400/600, Perdida: 0.05456
Epoca: 4/5, Paso: 500/600, Perdi

### Red Neuronal de 3 capas

#### Red Neuronal de 3 capas escondidas + Activaciones Sigmoide + SGD

In [9]:
RunModel("RN3", "Sigm", "SGD", 'runs/RN3_Sigm_SGD', saving_model = False)

Epoca: 1/5, Paso: 100/600, Perdida: 2.30880
Epoca: 1/5, Paso: 200/600, Perdida: 2.31254
Epoca: 1/5, Paso: 300/600, Perdida: 2.33314
Epoca: 1/5, Paso: 400/600, Perdida: 2.29346
Epoca: 1/5, Paso: 500/600, Perdida: 2.31503
Epoca: 1/5, Paso: 600/600, Perdida: 2.32177
Epoca: 2/5, Paso: 100/600, Perdida: 2.30833
Epoca: 2/5, Paso: 200/600, Perdida: 2.30469
Epoca: 2/5, Paso: 300/600, Perdida: 2.29137
Epoca: 2/5, Paso: 400/600, Perdida: 2.31299
Epoca: 2/5, Paso: 500/600, Perdida: 2.30627
Epoca: 2/5, Paso: 600/600, Perdida: 2.31453
Epoca: 3/5, Paso: 100/600, Perdida: 2.28361
Epoca: 3/5, Paso: 200/600, Perdida: 2.31213
Epoca: 3/5, Paso: 300/600, Perdida: 2.29823
Epoca: 3/5, Paso: 400/600, Perdida: 2.30357
Epoca: 3/5, Paso: 500/600, Perdida: 2.30616
Epoca: 3/5, Paso: 600/600, Perdida: 2.30596
Epoca: 4/5, Paso: 100/600, Perdida: 2.30053
Epoca: 4/5, Paso: 200/600, Perdida: 2.29734
Epoca: 4/5, Paso: 300/600, Perdida: 2.30434
Epoca: 4/5, Paso: 400/600, Perdida: 2.30561
Epoca: 4/5, Paso: 500/600, Perdi

#### Red Neuronal de 3 capas escondidas + Activaciones ReLU + SGD

In [10]:
RunModel("RN3", "ReLU", "SGD", 'runs/RN3_ReLU_SGD', saving_model = False)

Epoca: 1/5, Paso: 100/600, Perdida: 2.30752
Epoca: 1/5, Paso: 200/600, Perdida: 2.29760
Epoca: 1/5, Paso: 300/600, Perdida: 2.29984
Epoca: 1/5, Paso: 400/600, Perdida: 2.30616
Epoca: 1/5, Paso: 500/600, Perdida: 2.29596
Epoca: 1/5, Paso: 600/600, Perdida: 2.29809
Epoca: 2/5, Paso: 100/600, Perdida: 2.29873
Epoca: 2/5, Paso: 200/600, Perdida: 2.29051
Epoca: 2/5, Paso: 300/600, Perdida: 2.29435
Epoca: 2/5, Paso: 400/600, Perdida: 2.28852
Epoca: 2/5, Paso: 500/600, Perdida: 2.30028
Epoca: 2/5, Paso: 600/600, Perdida: 2.29430
Epoca: 3/5, Paso: 100/600, Perdida: 2.28674
Epoca: 3/5, Paso: 200/600, Perdida: 2.28457
Epoca: 3/5, Paso: 300/600, Perdida: 2.28827
Epoca: 3/5, Paso: 400/600, Perdida: 2.29361
Epoca: 3/5, Paso: 500/600, Perdida: 2.28675
Epoca: 3/5, Paso: 600/600, Perdida: 2.28899
Epoca: 4/5, Paso: 100/600, Perdida: 2.29439
Epoca: 4/5, Paso: 200/600, Perdida: 2.28527
Epoca: 4/5, Paso: 300/600, Perdida: 2.29050
Epoca: 4/5, Paso: 400/600, Perdida: 2.27703
Epoca: 4/5, Paso: 500/600, Perdi

#### Red Neuronal de 3 capas escondidas + Activaciones Sigmoide + Adam

In [11]:
RunModel("RN3", "Sigm", "Adam", 'runs/RN3_Sigm_Adam', saving_model = False)

Epoca: 1/5, Paso: 100/600, Perdida: 1.76503
Epoca: 1/5, Paso: 200/600, Perdida: 1.12979
Epoca: 1/5, Paso: 300/600, Perdida: 0.88725
Epoca: 1/5, Paso: 400/600, Perdida: 0.65549
Epoca: 1/5, Paso: 500/600, Perdida: 0.47046
Epoca: 1/5, Paso: 600/600, Perdida: 0.40680
Epoca: 2/5, Paso: 100/600, Perdida: 0.32816
Epoca: 2/5, Paso: 200/600, Perdida: 0.28952
Epoca: 2/5, Paso: 300/600, Perdida: 0.23769
Epoca: 2/5, Paso: 400/600, Perdida: 0.20549
Epoca: 2/5, Paso: 500/600, Perdida: 0.32464
Epoca: 2/5, Paso: 600/600, Perdida: 0.20629
Epoca: 3/5, Paso: 100/600, Perdida: 0.20500
Epoca: 3/5, Paso: 200/600, Perdida: 0.34998
Epoca: 3/5, Paso: 300/600, Perdida: 0.12600
Epoca: 3/5, Paso: 400/600, Perdida: 0.15098
Epoca: 3/5, Paso: 500/600, Perdida: 0.19190
Epoca: 3/5, Paso: 600/600, Perdida: 0.04021
Epoca: 4/5, Paso: 100/600, Perdida: 0.05183
Epoca: 4/5, Paso: 200/600, Perdida: 0.16688
Epoca: 4/5, Paso: 300/600, Perdida: 0.07373
Epoca: 4/5, Paso: 400/600, Perdida: 0.25256
Epoca: 4/5, Paso: 500/600, Perdi

#### Red Neuronal de 3 capas escondidas + Activaciones ReLU + Adam

In [12]:
RunModel("RN3", "ReLU", "Adam", 'runs/RN3_ReLU_Adam', saving_model = False)

Epoca: 1/5, Paso: 100/600, Perdida: 0.23658
Epoca: 1/5, Paso: 200/600, Perdida: 0.13085
Epoca: 1/5, Paso: 300/600, Perdida: 0.21071
Epoca: 1/5, Paso: 400/600, Perdida: 0.15012
Epoca: 1/5, Paso: 500/600, Perdida: 0.18413
Epoca: 1/5, Paso: 600/600, Perdida: 0.08857
Epoca: 2/5, Paso: 100/600, Perdida: 0.29261
Epoca: 2/5, Paso: 200/600, Perdida: 0.18313
Epoca: 2/5, Paso: 300/600, Perdida: 0.12010
Epoca: 2/5, Paso: 400/600, Perdida: 0.09019
Epoca: 2/5, Paso: 500/600, Perdida: 0.20964
Epoca: 2/5, Paso: 600/600, Perdida: 0.08635
Epoca: 3/5, Paso: 100/600, Perdida: 0.06402
Epoca: 3/5, Paso: 200/600, Perdida: 0.13483
Epoca: 3/5, Paso: 300/600, Perdida: 0.07353
Epoca: 3/5, Paso: 400/600, Perdida: 0.05996
Epoca: 3/5, Paso: 500/600, Perdida: 0.05578
Epoca: 3/5, Paso: 600/600, Perdida: 0.09890
Epoca: 4/5, Paso: 100/600, Perdida: 0.01791
Epoca: 4/5, Paso: 200/600, Perdida: 0.05250
Epoca: 4/5, Paso: 300/600, Perdida: 0.04896
Epoca: 4/5, Paso: 400/600, Perdida: 0.10034
Epoca: 4/5, Paso: 500/600, Perdi

### Red Neuronal de 5 capas

#### Red Neuronal de 5 capas escondidas + Activaciones Sigmoide + SGD

In [13]:
RunModel("RN5", "Sigm", "SGD", 'runs/RN5_Sigm_SGD', saving_model = False)

Epoca: 1/5, Paso: 100/600, Perdida: 2.34314
Epoca: 1/5, Paso: 200/600, Perdida: 2.28729
Epoca: 1/5, Paso: 300/600, Perdida: 2.31738
Epoca: 1/5, Paso: 400/600, Perdida: 2.33431
Epoca: 1/5, Paso: 500/600, Perdida: 2.29466
Epoca: 1/5, Paso: 600/600, Perdida: 2.29585
Epoca: 2/5, Paso: 100/600, Perdida: 2.31283
Epoca: 2/5, Paso: 200/600, Perdida: 2.30929
Epoca: 2/5, Paso: 300/600, Perdida: 2.31403
Epoca: 2/5, Paso: 400/600, Perdida: 2.28551
Epoca: 2/5, Paso: 500/600, Perdida: 2.31438
Epoca: 2/5, Paso: 600/600, Perdida: 2.29786
Epoca: 3/5, Paso: 100/600, Perdida: 2.29774
Epoca: 3/5, Paso: 200/600, Perdida: 2.30407
Epoca: 3/5, Paso: 300/600, Perdida: 2.30064
Epoca: 3/5, Paso: 400/600, Perdida: 2.30033
Epoca: 3/5, Paso: 500/600, Perdida: 2.31409
Epoca: 3/5, Paso: 600/600, Perdida: 2.30282
Epoca: 4/5, Paso: 100/600, Perdida: 2.30571
Epoca: 4/5, Paso: 200/600, Perdida: 2.29104
Epoca: 4/5, Paso: 300/600, Perdida: 2.30091
Epoca: 4/5, Paso: 400/600, Perdida: 2.29528
Epoca: 4/5, Paso: 500/600, Perdi

#### Red Neuronal de 5 capas escondidas + Activaciones ReLU + SGD

In [14]:
RunModel("RN5", "ReLU", "SGD", 'runs/RN5_ReLU_SGD', saving_model = False)

Epoca: 1/5, Paso: 100/600, Perdida: 2.29845
Epoca: 1/5, Paso: 200/600, Perdida: 2.30885
Epoca: 1/5, Paso: 300/600, Perdida: 2.29811
Epoca: 1/5, Paso: 400/600, Perdida: 2.31248
Epoca: 1/5, Paso: 500/600, Perdida: 2.30376
Epoca: 1/5, Paso: 600/600, Perdida: 2.30326
Epoca: 2/5, Paso: 100/600, Perdida: 2.30227
Epoca: 2/5, Paso: 200/600, Perdida: 2.30039
Epoca: 2/5, Paso: 300/600, Perdida: 2.31407
Epoca: 2/5, Paso: 400/600, Perdida: 2.30311
Epoca: 2/5, Paso: 500/600, Perdida: 2.30233
Epoca: 2/5, Paso: 600/600, Perdida: 2.30865
Epoca: 3/5, Paso: 100/600, Perdida: 2.30527
Epoca: 3/5, Paso: 200/600, Perdida: 2.29979
Epoca: 3/5, Paso: 300/600, Perdida: 2.31136
Epoca: 3/5, Paso: 400/600, Perdida: 2.30690
Epoca: 3/5, Paso: 500/600, Perdida: 2.31134
Epoca: 3/5, Paso: 600/600, Perdida: 2.30824
Epoca: 4/5, Paso: 100/600, Perdida: 2.30211
Epoca: 4/5, Paso: 200/600, Perdida: 2.30509
Epoca: 4/5, Paso: 300/600, Perdida: 2.31265
Epoca: 4/5, Paso: 400/600, Perdida: 2.30074
Epoca: 4/5, Paso: 500/600, Perdi

#### Red Neuronal de 5 capas escondidas + Activaciones Sigmoide + Adam

In [15]:
RunModel("RN3", "Sigm", "Adam", 'runs/RN5_Sigm_Adam', saving_model = False)

Epoca: 1/5, Paso: 100/600, Perdida: 1.75372
Epoca: 1/5, Paso: 200/600, Perdida: 1.22390
Epoca: 1/5, Paso: 300/600, Perdida: 0.86268
Epoca: 1/5, Paso: 400/600, Perdida: 0.72304
Epoca: 1/5, Paso: 500/600, Perdida: 0.40169
Epoca: 1/5, Paso: 600/600, Perdida: 0.43360
Epoca: 2/5, Paso: 100/600, Perdida: 0.26429
Epoca: 2/5, Paso: 200/600, Perdida: 0.23229
Epoca: 2/5, Paso: 300/600, Perdida: 0.30579
Epoca: 2/5, Paso: 400/600, Perdida: 0.22136
Epoca: 2/5, Paso: 500/600, Perdida: 0.20515
Epoca: 2/5, Paso: 600/600, Perdida: 0.08642
Epoca: 3/5, Paso: 100/600, Perdida: 0.18626
Epoca: 3/5, Paso: 200/600, Perdida: 0.19002
Epoca: 3/5, Paso: 300/600, Perdida: 0.22303
Epoca: 3/5, Paso: 400/600, Perdida: 0.13499
Epoca: 3/5, Paso: 500/600, Perdida: 0.20281
Epoca: 3/5, Paso: 600/600, Perdida: 0.26274
Epoca: 4/5, Paso: 100/600, Perdida: 0.06373
Epoca: 4/5, Paso: 200/600, Perdida: 0.06455
Epoca: 4/5, Paso: 300/600, Perdida: 0.11866
Epoca: 4/5, Paso: 400/600, Perdida: 0.14202
Epoca: 4/5, Paso: 500/600, Perdi

#### Red Neuronal de 5 capas escondidas + Activaciones ReLU + Adam

In [16]:
RunModel("RN3", "ReLU", "Adam", 'runs/RN5_ReLU_Adam', saving_model = False)

Epoca: 1/5, Paso: 100/600, Perdida: 0.47554
Epoca: 1/5, Paso: 200/600, Perdida: 0.27499
Epoca: 1/5, Paso: 300/600, Perdida: 0.21106
Epoca: 1/5, Paso: 400/600, Perdida: 0.23222
Epoca: 1/5, Paso: 500/600, Perdida: 0.25960
Epoca: 1/5, Paso: 600/600, Perdida: 0.18393
Epoca: 2/5, Paso: 100/600, Perdida: 0.12968
Epoca: 2/5, Paso: 200/600, Perdida: 0.12914
Epoca: 2/5, Paso: 300/600, Perdida: 0.17494
Epoca: 2/5, Paso: 400/600, Perdida: 0.23223
Epoca: 2/5, Paso: 500/600, Perdida: 0.06750
Epoca: 2/5, Paso: 600/600, Perdida: 0.07224
Epoca: 3/5, Paso: 100/600, Perdida: 0.05423
Epoca: 3/5, Paso: 200/600, Perdida: 0.08579
Epoca: 3/5, Paso: 300/600, Perdida: 0.05958
Epoca: 3/5, Paso: 400/600, Perdida: 0.22338
Epoca: 3/5, Paso: 500/600, Perdida: 0.04557
Epoca: 3/5, Paso: 600/600, Perdida: 0.05569
Epoca: 4/5, Paso: 100/600, Perdida: 0.14058
Epoca: 4/5, Paso: 200/600, Perdida: 0.04078
Epoca: 4/5, Paso: 300/600, Perdida: 0.10372
Epoca: 4/5, Paso: 400/600, Perdida: 0.10707
Epoca: 4/5, Paso: 500/600, Perdi